<a href="https://colab.research.google.com/github/KangHyeonGu1/-/blob/main/solar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
data = pd.read_csv("solar data.txt")
data.head

<bound method NDFrame.head of           temp  cloud cover      wind  humidity  barometer  day_power
0     0.312052     0.142857  0.234784  0.778769   0.638520   0.058824
1     0.369939     0.285714  0.412924  0.725910   0.424439   0.058824
2     0.352761     0.571429  0.572187  0.709644   0.296313   0.058824
3     0.348520     0.285714  0.334605  0.751113   0.118489   0.029412
4     0.348466     0.714286  0.313384  0.735478   0.160241   0.029412
...        ...          ...       ...       ...        ...        ...
1327  0.263113     0.714286  0.268356  0.786389   0.318577   0.058824
1328  0.252461     0.714286  0.105177  0.762446   0.423438   0.117647
1329  0.257106     0.142857  0.103314  0.809187   0.474831   0.088235
1330  0.326789     0.857143  0.292398  0.759844   0.455705   0.000000
1331  0.287142     0.428571  0.232619  0.681145   0.565277   0.176471

[1332 rows x 6 columns]>

In [73]:
# 독립변수 기온 spread설정과 삼각퍼지화
te=data['temp']
spread= [None] * 1332
for i in range(0,1331):
  spread[i]= (np.abs(te[i+1]-te[i]))/2
spread[1331]=spread[1330]
x1= te-spread
x2= te+spread
xdf=pd.DataFrame({'a':x1,'b':te,'c':x2})
x=xdf.to_numpy()


In [77]:
# 매개변수 습도 spread설정과 삼각퍼지화. 0-1사이로 정규화 하였기 때문에
# 일단 기온과 같이 spread화 했지만 생각할 필요성이 있음. 
hu=data['humidity']
spreadm= [None] * 1332
for i in range(0,1331):
  spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
spreadm[1331]=spreadm[1330]
m1= hu-spreadm
m2= hu+spreadm
mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
m=mdf.to_numpy()

array([[0.75233957, 0.77876915, 0.80519874],
       [0.71777698, 0.72590998, 0.73404298],
       [0.68890934, 0.70964398, 0.73037862],
       ...,
       [0.78451541, 0.80918683, 0.83385826],
       [0.72049462, 0.75984398, 0.79919334],
       [0.6417959 , 0.68114526, 0.72049462]])